# Correlation Analysis

In this article, things that I would like to acheive are listed below.

1. Get the marketcap weighted portfolio
2. Get a list of ETFs from the market
3. Perform a correlation analysis
    - A heatmap on the panel
    - A moving correlation
4. Copula 
5. Efficient Frontier (Or just standard deviation and returns)

## Marketcap weighted portfolio of Cryptos

In [108]:
import pandas as pd
import datetime 
import numpy as np
import time

import matplotlib.pyplot as plt
%matplotlib inline

# =================
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [112]:
names = ['bitcoin_cash', 'bitcoin', 'bitconnect', 'dash', 'ethereum', 'ethereum_classic', 'iota', 'litecoin', 
         'monero', 'nem', 'neo', 'numeraire', 'ripple', 'stratis', 'waves']
file_n = [a + '_price.csv' for a in names]
df = pd.DataFrame()
for f in range(len(file_n)):
    tdf = pd.read_csv('/Users/HoumanDehghan/Documents/Cryptos/' + file_n[f])
    tdf = tdf.iloc[:,[0, 6, 4]]
    tdf['Coin'] = names[f]
    dates = pd.to_datetime(tdf.Date, format = '%b %d, %Y')
    dates.apply(lambda x: x.strftime('%Y-%m-%d'))
    tdf['Date'] = dates
    df = df.append(tdf, ignore_index=True)
    
all_prices = df.pivot(index='Date', columns='Coin', values='Close')
def floater(x):
    if type(x) is int:
        return float(x)
    else:
        s_out = x.replace(',', '').replace('-','').replace(' ','')
        if len(s_out) == 0:
            return float(0)
        else:
            return float(s_out)
        
all_caps = df.pivot(index='Date', columns='Coin', values='Market Cap')
all_caps.fillna(value = 0, inplace = True)
all_caps = all_caps.applymap(floater)
weights = all_caps.sum(axis = 1)
weights = all_caps.div(weights, axis="index")
all_rets = all_prices.pct_change()
all_rets.fillna(value = 0, inplace = True)
# Get the performance of a cap weighted fund
# Compare it with SPX
real_rets = all_rets.shift(-1) * weights
real_rets = real_rets.sum(axis=1)
pnl = (real_rets + 1).cumprod() 

In [113]:
# Plot the PnL, save it
# Chart the BTC pricing data
# Plot
cap_crypto = go.Scatter(x=pnl.index, y=pnl)
# Layout
layout= go.Layout(
    title= 'Equity Curve of a $1 Invested in Cap-weighted Portfolio',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Date',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Value of a $1 Invested',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend= False
)
fig = go.Figure(data = [cap_crypto], layout = layout)
py.iplot(fig)

## A List of ETFs

In [4]:
import IB_PY as ibpy
import datetime as datetime
from alpha_vantage.timeseries import TimeSeries

In [104]:
# Getting the data from AlphaVantage
# API : 4SO00AO046LKGPHB
ts = TimeSeries(key='4SO00AO046LKGPHB', output_format='pandas')


In [109]:
# Merging all the data
all_data, metadata = ts.get_daily_adjusted('SPY', outputsize = 'full')
all_data = all_data.loc[:,'close']
tickers = ['XLE', 'XLU', 'XLK', 'XLB', 'XLP', 'XLV', 'XLF', 'AGG', 'GSG', 'HYG', 'VNQ', 'VIX']

for ticker in tickers:
    time.sleep(1)
    data, metadata = ts.get_daily_adjusted(ticker, outputsize = 'full')
    all_data = pd.concat([all_data, data.loc[:, 'close']], axis=1, join='inner')
    
all_data.columns = ['SPY', 'XLE', 'XLU', 'XLK', 'XLB', 'XLP', 'XLV', 'XLF', 'AGG', 'GSG', 'HYG', 'VNQ', 'VIX']

In [110]:
all_data.head()
#all_data.SPY.plot()
#all_data.plot()

,SPY,XLE,XLU,XLK,XLB,XLP,XLV,XLF,AGG,GSG,HYG,VNQ,VIX
Date,,,,,,,,,,,,,
2007-04-11,144.02,62.07,40.91,23.62,38.45,26.97,34.53,35.61,99.63,41.45,104.35,79.35,13.49
2007-04-12,144.66,63.02,40.73,23.87,38.79,27.00,34.92,35.55,99.75,41.75,104.42,78.81,12.71
2007-04-13,145.32,63.05,40.72,23.89,38.88,27.09,35.43,35.70,99.59,41.80,104.23,79.70,12.20
2007-04-16,146.70,63.42,40.96,24.09,39.40,27.15,35.77,36.57,99.79,41.52,104.19,79.80,11.98
2007-04-17,147.09,63.06,41.15,24.14,39.03,27.40,35.88,36.56,100.08,41.16,104.14,80.84,12.14


In [114]:
pnl.index.name = 'Date'
pnl.name = 'CRP'
pnl.index = pnl.index.strftime('%Y-%m-%d')
all_df = pd.concat([pnl, all_data], join = 'inner', axis = 1)

In [115]:
all_df.head()

,CRP,SPY,XLE,XLU,XLK,XLB,XLP,XLV,XLF,AGG,GSG,HYG,VNQ,VIX
2013-04-29,1.033563,159.300,77.96,41.32,30.47,39.49,41.02,47.69,18.65,111.56,31.53,95.5500,74.41,13.71
2013-04-30,0.871871,159.680,78.27,41.43,30.80,39.55,40.94,47.33,18.70,111.54,31.29,95.8500,75.28,13.52
2013-05-01,0.783583,158.280,77.05,41.02,30.53,38.85,40.90,46.83,18.49,111.39,30.64,95.2000,74.68,14.49
2013-05-02,0.727005,159.750,78.06,40.97,30.92,39.13,41.05,47.35,18.65,111.40,31.24,95.8101,75.17,13.59
2013-05-03,0.836495,161.368,79.47,40.89,31.21,39.82,41.23,47.67,18.85,110.99,31.51,95.9600,75.47,12.85


In [116]:
# Filtering bad data
all_df_ret = all_df.pct_change()
#all_df_ret[np.abs(all_df_ret) > 0.5] = np.nan
# Filtering for returns more than 50% 
# That is just insane, really bad data

In [118]:
def correlation_heatmap(df, title, absolute_bounds=True, method_ = 'pearson'):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method=method_).as_matrix(),
        x=df.columns,
        y=df.columns,
        colorbar=dict(title= method_ + ' coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [119]:
correlation_heatmap(all_df_ret, "Cryptocurrency and the Markets")

In [120]:
correlation_heatmap(all_df_ret, "Cryptocurrency and the Markets", method_='spearman')

In [121]:
correlation_heatmap(all_df_ret, "Cryptocurrency and the Markets", method_='kendall')

In [122]:
roll_corr = pd.rolling_corr(all_df_ret.iloc[:,0], all_df_ret.iloc[:,1:], window=60)

/Users/HoumanDehghan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning:

pd.rolling_corr is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=60).corr(other=<DataFrame>)



In [123]:
ts = []
N = len(roll_corr.columns)
for c in roll_corr.columns[:N/3]:
    ts.append(go.Scatter(x = roll_corr.index, y = roll_corr.loc[:,c], mode = 'line', name = c))

data = ts
py.iplot(data)

In [124]:
ts = []
N = len(roll_corr.columns)
for c in roll_corr.columns[N/3:2*N/3]:
    ts.append(go.Scatter(x = roll_corr.index, y = roll_corr.loc[:,c], mode = 'line', name = c))

data = ts
py.iplot(data)

In [125]:
ts = []
N = len(roll_corr.columns)
for c in roll_corr.columns[2*N/3:]:
    ts.append(go.Scatter(x = roll_corr.index, y = roll_corr.loc[:,c], mode = 'line', name = c))

data = ts
py.iplot(data)

In [126]:
all_df_ret.mean(axis = 0) * 25200, all_df_ret.std(axis = 0) * np.sqrt(252) * 100

(CRP    121.875691
 SPY     10.671107
 XLE     -2.936740
 XLU      8.586215
 XLK     15.905097
 XLB      8.577481
 XLP      7.320346
 XLV     12.964904
 XLF      8.051912
 AGG     -0.198444
 GSG    -16.696323
 HYG     -1.639060
 VNQ      3.716506
 VIX     69.821853
 dtype: float64, CRP     82.418673
 SPY     12.311790
 XLE     20.527482
 XLU     19.782837
 XLK     14.393483
 XLB     15.840570
 XLP     11.696525
 XLV     15.332095
 XLF     18.118886
 AGG      3.443035
 GSG     18.988469
 HYG      6.744013
 VNQ     15.300573
 VIX    124.597656
 dtype: float64)

In [127]:
# Plotting the risk return profile of this
# We do a scatter plot of mean return vs standard deviation
# Create a trace
trace = go.Scatter(
    x = all_df_ret.std(axis = 0) * np.sqrt(252) * 100,
    y = all_df_ret.mean(axis = 0) * 252 * 100,
    text = all_df_ret.columns,
    textposition = 'top',
    mode = 'markers+text'
)

# Layout
layout= go.Layout(
    title= 'Cryptos Risk-return Trade-off',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Annualized Volatility (standard deviation)',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Annualized Return',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend= False
)

data = [trace]

fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

In [68]:
# Statistics on SPY and CRP
print all_df_ret.mean(axis = 0) * 25200

CRP    121.875691
SPY     10.671107
XLE     -2.936740
XLU      8.586215
XLK     15.905097
XLB      8.577481
XLP      7.320346
XLV     12.964904
XLF      8.051912
AGG     -0.198444
GSG    -16.696323
dtype: float64


In [69]:
# Annualized Volatility
print all_df_ret.std(axis = 0) * np.sqrt(252) * 100

CRP    82.418673
SPY    12.311790
XLE    20.527482
XLU    19.782837
XLK    14.393483
XLB    15.840570
XLP    11.696525
XLV    15.332095
XLF    18.118886
AGG     3.443035
GSG    18.988469
dtype: float64


In [129]:
# Largest Drawdown
# Longest Drawdown
# SPY and CRP
def create_drawdowns(equity_curve):
    """
    Calculate the largest peak-to-trough drawdown of the PnL curve
    as well as the duration of the drawdown. Requires that the 
    pnl_returns is a pandas Series.
    
    Taken from QuantStart - https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-VII

    Parameters:
    pnl - A pandas Series representing period percentage returns.

    Returns:
    drawdown, duration - Highest peak-to-trough drawdown and duration.
    """

    # Calculate the cumulative returns curve 
    # and set up the High Water Mark
    # Then create the drawdown and duration series
    hwm = [0]
    eq_idx = equity_curve.index
    drawdown = pd.Series(index = eq_idx)
    # duration = pd.Series(index = eq_idx)

    # Loop over the index range
    for t in range(1, len(eq_idx)):
        cur_hwm = max(hwm[t-1], equity_curve[t])
        hwm.append(cur_hwm)
        drawdown[t]= hwm[t] - equity_curve[t]
        # duration[t]= 0 if drawdown[t] == 0 else duration[t-1] + 1
    return drawdown

In [130]:
DD_CRP = create_drawdowns(all_df.loc[:,'CRP'])
DD_SPY = create_drawdowns(all_df.loc[:,'SPY'])
DD = pd.concat([-1 * DD_CRP, -1 * DD_SPY], axis=1, join='inner')
DD.columns = ['CRP', 'SPY']

In [131]:
ts = []
N = len(DD.columns)
for c in DD.columns:
    ts.append(go.Scatter(x = DD.index, y = DD.loc[:,c], mode = 'line', name = c))

data = ts
py.iplot(data)

## Copula

Correlation is not a good measure of codependence. So let's look at their copulas. 
Meucci. 

In [132]:
from statsmodels.distributions.empirical_distribution import ECDF

In [133]:
ecdf1 = ECDF(all_df_ret.loc[:,'SPY'])
ecdf2 = ECDF(all_df_ret.loc[:,'CRP'])

In [134]:
points_x = ecdf2(all_df_ret.loc[:,'CRP'])
points_y = ecdf1(all_df_ret.loc[:,'SPY'])

In [135]:
# Create a trace
trace = go.Scatter(
    x = points_x,
    y = points_y,
    mode = 'markers'
)

data = [trace]

layout= go.Layout(
    title= 'Co-dependence of Market-capitalization Weighted Portfolios',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Inverse Empirical CDF of CRP Returns',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Inverse Empirical CDF of SPY Returns',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend= False
)

# Plot and embed in ipython notebook!
fig = go.Figure(data = data, layout = layout)
py.iplot(fig)